In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import pyodbc
import adodbapi as ado
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Ограничения на % выполнения от выпуска:

In [2]:
i_f_w=95
i_other=98

# Загрузка и предобработка PPS с учетом ограничений

In [4]:
%%time
#Загружаем основную таблицу PPS

server = 'private data'
database = 'private data'
connection = pyodbc.connect('private data')



sql = """select 
VpextM_P_PS.model as model,
VpextM_P_PS.DT as DT, 
VpextM_P_PS.num,
PodrsExt.name, PodrsExt.ros, 
modelsall.modeliN, 
modelsall.zakaz,  modelsall.model,
DepartmentS.Name as Dep,
CategoryS.Name as Cat,
SubCategoryS.Name as subCat,
typsRazb.num as razb,
VpextM_P_PS.Vp,
VpextM_P_PS.minutVp


from VpextM_P_PS

left join PodrsExt on VpextM_P_PS.podr = PodrsExt.podr
left join modelsall on VpextM_P_PS.model = modelsall.model
left join typsRazb on modelsall.razb = typsRazb.num
left join DepartmentS on modelsall.Department = DepartmentS.Department
left join CategoryS on modelsall.Category = CategoryS.Category
left join SubCategoryS on modelsall.SubCategory = SubCategoryS.SubCategory




where year(VpextM_P_PS.DT)>=2021
"""
cursor = connection.cursor()
Vpext = cursor.execute(sql)
Vpext = pd.DataFrame.from_records(Vpext.fetchall(),
columns=[column[0] for column in Vpext.description])


Vpext=Vpext.dropna(subset=['modeliN'])
Vpext['StyleNo']=Vpext.modeliN.str.split(' ', expand = True)[0]
Vpext=Vpext.drop(['model'], axis=1)

Wall time: 4.01 s


In [5]:
#Определение процентов

percent_oper=pd.DataFrame({'name':Vpext.name.unique()})
percent_oper['percent']=np.where(((percent_oper=='Finishing')|(percent_oper=='Washing')), i_f_w, i_other)

In [6]:
#Убираем из данных строки, где Vp и minutVp не нули

Vpext=Vpext[~((Vpext.Vp==0)&(Vpext.minutVp==0))]
Vpext = Vpext.drop(['minutVp'], axis=1)
Vpext=Vpext.reset_index(drop=True)

# Загрузка куба

In [7]:
%%time
# Запрос к кубу

conn_args = {'timeout' : 90000}
with ado.connect('''private data''',  conn_args) as con:

        with con.cursor() as cur:
            first_mdx = ''' SELECT { [Measures].[Выпуск шт] } ON COLUMNS,
            { ([Модели].[Модель].[Модель].ALLMEMBERS
            * [Модели].[Тип производства].[Тип производства].ALLMEMBERS
            * [Модели].[Новые ЦГ 0-8].[Новые ЦГ 0-8].ALLMEMBERS) } 
            DIMENSION PROPERTIES MEMBER_CAPTION,
            MEMBER_UNIQUE_NAME ON ROWS FROM [SalesChecksCustomers] '''


            # all model

            cur.execute(first_mdx)
            data0 = cur.fetchall()
            ar = np.array(data0.ado_results)
            df = pd.DataFrame(ar).transpose()
            df.columns = data0.columnNames.keys()

df = df[[col for col in df.columns if 'unique' not in col]]
df.columns = ['modeliN', 'Тип','Target group','Output, Units']
df=df.drop(['Output, Units'], axis=1)

Wall time: 31.6 s


# Загрузка PLM

In [8]:
#PLM

server = 'private data'
database = 'private data'
connection = pyodbc.connect('private data')



sql = """SELECT *
         FROM vwx_GJ_ProductionPlanning_CC_SEL
     """

cursor = connection.cursor()
plm_base_cc_raw = cursor.execute(sql)
plm_base_cc = pd.DataFrame.from_records(plm_base_cc_raw.fetchall(),
                                             columns=[column[0] for column in plm_base_cc_raw.description])

cursor.close()

In [9]:
#Берем из PLM только ДНП

plm_base_cc=plm_base_cc[['ДНП','StyleNo']]
plm_base_cc = plm_base_cc.rename(columns = {'ДНП': 'dnp'}, inplace = False)

# Соединение PPS и PLM

In [10]:
#Присоединяем ДНП из PLM к осн. таблице

Vpext=Vpext.merge(plm_base_cc, how = 'left', on ='StyleNo' )

In [11]:
#Присоединяем ДНП из куба к осн. таблице

Vpext=Vpext.merge(df, how = 'left', on ='modeliN')

In [12]:
#Только основное производство

Vpext = Vpext.loc[(Vpext.razb.isna())|(Vpext.razb==1)]
Vpext=Vpext.drop(['razb'], axis=1)
Vpext=Vpext.reset_index(drop=True)

In [14]:
Vpext.isna().sum()

DT               0
num              0
name             0
ros              0
modeliN          0
zakaz            0
Dep              0
Cat              0
subCat           0
Vp               0
StyleNo          0
dnp             90
Тип             82
Target group    82
dtype: int64

# Загрузка ОДВ

In [15]:
%%time
#Загружаем ОДВ

load_odv=pd.read_excel('Одв.xlsx')

Wall time: 2min 25s


In [16]:
#Оставляем только модель и статус
odv= load_odv[['Style', 'Order status', 'Category Production','Production','current season']]
odv['StyleNo'] = odv['Style'].str.split(' ', expand = True)[0]
odv = odv.rename(columns = {'Style': 'modeliN'}, inplace = False)
odv = odv.reset_index(drop=True)

In [17]:
#Оставляем только импортные строчки без отмены на модель

odv = odv[~((odv.Production == 'import')|(odv['current season'].str.contains('отмена')))]
odv = odv.reset_index(drop=True)

# Соединение всех 3-х баз

In [18]:
#Присоединяем ОДВ к Vpext, фильтруем статус, убираем лишние столбики

pps_odv_plm = Vpext.merge(odv[['StyleNo', 'Order status', 'Category Production']], how = 'left', on ='StyleNo' )
pps_odv_plm=pps_odv_plm[pps_odv_plm['Order status']=='закрыт']
pps_odv_plm = pps_odv_plm.drop(['Order status'], axis=1)
pps_odv_plm = pps_odv_plm.reset_index(drop=True)

# Обработка данных с учетом ограничений на % выпуска от заказа

In [19]:
#Считаем накопленную сумму выпуска по каждой операции модели

cumul_Vp=pps_odv_plm.groupby(['modeliN', 'name', 'DT'])['Vp'].sum().groupby(level=[0,1]).cumsum().reset_index()
cumul_Vp=cumul_Vp.rename(columns = {'Vp': 'Vp_cumul'}, inplace = False)

In [20]:
#Считаем % выполнения заказа 

cumul_percent = pps_odv_plm.merge(cumul_Vp, how='left', on=['modeliN', 'name', 'DT'])
cumul_percent['%Vp'] = cumul_percent['Vp_cumul']/cumul_percent['zakaz']*100
cumul_percent = cumul_percent.merge(percent_oper, how='left', on='name')

In [21]:
#Таблица для идентификации строк, превышающих % выпуска

cumul_percent['is_del']=np.where(cumul_percent['%Vp']>=cumul_percent.percent,1,0)
cumul_percent_del=cumul_percent.groupby(['modeliN', 'name', 'DT'])['is_del'].sum().groupby(level=[0,1]).cumsum().reset_index()
cumul_percent_del=cumul_percent_del.rename(columns = {'is_del': 'is_del_cumul'}, inplace = False)

#Данные без превышения % выпуска
clear_cumul_data = cumul_percent.merge(cumul_percent_del, how='left', on=['modeliN', 'name', 'DT'])

In [22]:
#Оставляем только те строки, где % выпуска не превышен

pps_odv_plm=clear_cumul_data[clear_cumul_data.is_del==clear_cumul_data.is_del_cumul]
pps_odv_plm=pps_odv_plm.drop(['Vp_cumul','%Vp','is_del','percent','is_del_cumul'], axis=1)

In [23]:
#Список моделей, которые производятся у субкотракторов
subcontr_model_list = list(pps_odv_plm[pps_odv_plm.ros=='Субконтракторы                '].modeliN.unique())

In [24]:
#Список моделей, производство которых было и в России, и в Украине

data_rus_and_ukr = pps_odv_plm[(pps_odv_plm.ros=='Россия                        ')|
            (pps_odv_plm.ros=='Украина                       ')][['modeliN','ros']].drop_duplicates().groupby(
                                                               'modeliN').count().reset_index()
data_rus_and_ukr = data_rus_and_ukr[data_rus_and_ukr.ros>1]

rus_and_ukr_list = list(data_rus_and_ukr.modeliN.unique())

In [25]:
#Замена стран

pps_odv_plm.loc[pps_odv_plm.modeliN.isin(subcontr_model_list), 'ros'] = 'Субконтракторы'
pps_odv_plm.loc[pps_odv_plm.modeliN.isin(rus_and_ukr_list), 'ros'] = 'Россия+Украина' 


# Обработка данных с учтом ограничений на даты ДНП и последней операции

In [26]:
#Оставляем только те модели, ДНП и последняя дата операции которых не раньше 1го июля

data_cur_year=pd.DataFrame(pps_odv_plm.groupby(['modeliN','dnp'])['DT'].max().reset_index())
new_data=data_cur_year[data_cur_year.DT >= '2021-07-01 00:00:00']
new_data=new_data[data_cur_year.dnp >= '2021-07-01 00:00:00']

#В основной таблице оставляем отфильтрованные данные
pps_odv_plm=pps_odv_plm.drop(['dnp'], axis=1)
new_data = new_data.drop(['DT'], axis=1)
pps_odv_plm = new_data.merge(pps_odv_plm, how = 'left', on ='modeliN')
pps_odv_plm = pps_odv_plm.reset_index(drop=True)

# Подсчет дней

In [27]:
#Total production by countries

dtt = pps_odv_plm.sort_values(['DT'], ascending=False).groupby(['ros','modeliN'])['DT']

total_time_data = pd.DataFrame((dtt.first() - dtt.last()).dt.days)
total_time_data['Days'] = total_time_data.DT+1
total_time_data=total_time_data.drop(['DT'], axis=1)

total_time_data['name'] = 'Production Total'
total_time_data = total_time_data.merge(pps_odv_plm[['modeliN','zakaz','dnp','Dep','Cat','subCat','ros','Тип','Target group','Category Production']], how = 'left', on = ['modeliN', 'ros'])
total_time_data = total_time_data[['ros','name','modeliN','Days','dnp','zakaz','Dep','Cat','subCat', 'Тип','Target group','Category Production' ]]
total_time_data = total_time_data.drop_duplicates()
total_time_data=total_time_data.reset_index(drop=True)

In [28]:
#pre-Sewing days

pre_sewing_list = ['Cutting', 'Embroidery','Printing','Automated Machines']

pre_dts=pps_odv_plm[pps_odv_plm.name.isin(pre_sewing_list)].sort_values(
            ['DT'], ascending=False).groupby(['ros','modeliN'])['DT']

pre_sewing_data = pd.DataFrame((pre_dts.first() - pre_dts.last()).dt.days)
pre_sewing_data['Days'] = pre_sewing_data.DT+1
pre_sewing_data=pre_sewing_data.drop(['DT'], axis=1)

pre_sewing_data['name'] = 'pre-Sewing'
pre_sewing_data = pre_sewing_data.merge(pps_odv_plm[['modeliN','zakaz','dnp','Dep','Cat','subCat','ros','Тип','Target group','Category Production']], how = 'left', on = ['modeliN', 'ros'])
pre_sewing_data = pre_sewing_data[['ros','name','modeliN','Days','dnp','zakaz','Dep','Cat','subCat', 'Тип','Target group','Category Production' ]]
pre_sewing_data = pre_sewing_data.drop_duplicates()
pre_sewing_data=pre_sewing_data.reset_index(drop=True)

In [29]:
#post-Sewing days

post_sewing_list = ['Post Sewing','Washing','SE-Attrition','SE-Brush','SE-Glove','SE-Pinks', 'SE-Silicone',
                    'SE-Spray','SE-Laser','SE-Mesh','Finishing']

post_dts=pps_odv_plm[pps_odv_plm.name.isin(post_sewing_list)].sort_values(
            ['DT'], ascending=False).groupby(['ros','modeliN'])['DT']

post_sewing_data = pd.DataFrame((post_dts.first() - post_dts.last()).dt.days)
post_sewing_data['Days'] = post_sewing_data.DT+1
post_sewing_data=post_sewing_data.drop(['DT'], axis=1)

post_sewing_data['name'] = 'post-Sewing'
post_sewing_data = post_sewing_data.merge(pps_odv_plm[['modeliN','zakaz','dnp','Dep','Cat','subCat','ros','Тип','Target group','Category Production']], how = 'left', on = ['modeliN', 'ros'])
post_sewing_data = post_sewing_data[['ros','name','modeliN','Days','dnp','zakaz','Dep','Cat','subCat', 'Тип','Target group','Category Production' ]]
post_sewing_data = post_sewing_data.drop_duplicates()
post_sewing_data=post_sewing_data.reset_index(drop=True)

In [30]:
#Total production for drop_list

total_without_ros = pps_odv_plm.sort_values(['DT'], ascending=False).groupby(['modeliN'])['DT']
total_model_data = pd.DataFrame((total_without_ros.first() - total_without_ros.last()).dt.days)
total_model_data['Days'] = total_model_data.DT+1
total_model_data=total_model_data.drop(['DT'], axis=1)


#Созание списка с моделями, total по которым меньше 80% и не больше 100 дней
drop_model_list=total_model_data[((total_model_data.Days>=total_model_data.Days.quantile(q=0.8, interpolation='nearest'))|
                                     (total_model_data.Days>100)| (total_model_data.Days<8))]

drop_model_list = drop_model_list.reset_index()
drop_model_list = list(drop_model_list.modeliN.values)
drop_model_list = list(set(drop_model_list))

In [31]:
#Удаляем модели, по которым не было пошива или кройки

no_name_data = pps_odv_plm[(pps_odv_plm.name=="Sewing")|
                           (pps_odv_plm.name=='Cutting')][['modeliN', 'name']].drop_duplicates().groupby(
                                                               'modeliN').count().reset_index()
no_name_data = no_name_data[no_name_data.name<2]
no_name_list = list(no_name_data.modeliN.unique())

for i in no_name_list:
    if i in drop_model_list:
        no_name_list.remove(i)

pps_odv_plm = pps_odv_plm.loc[~(pps_odv_plm.modeliN.isin(no_name_list))]

In [32]:
#Группируем данные, сортируя по дате
dt = pps_odv_plm.sort_values(['DT'], ascending=False).groupby(['ros','name','modeliN'])['DT']


#Считаем количество дней
time_data = pd.DataFrame((dt.first() - dt.last()).dt.days)
time_data['Days'] = time_data.DT+1
time_data=time_data.drop(['DT'], axis=1)

In [33]:
#Добавляем столбик с днями по операциям к итоговой таблице

pps_odv_plm=pps_odv_plm.drop(['DT','num','StyleNo'], axis=1)
pps_odv_plm=pps_odv_plm.drop_duplicates()

#Итоговая таблица для загрузки
result=time_data.merge(pps_odv_plm, how = 'left', on = ['ros','name','modeliN'])

In [34]:
#Присоединяем строчки с тоталом с итоговой таблице

result = pd.concat([result, total_time_data], ignore_index=True) 
result = pd.concat([result, pre_sewing_data], ignore_index=True)
result = pd.concat([result, post_sewing_data], ignore_index=True)
result = result.drop(['Vp'],axis=1)
result = result.drop_duplicates()

# Операции в столбиках (Pivot)

In [35]:
#Разбиваем операции на столбики

pivot = pd.pivot_table(result, values='Days', index=['modeliN','ros'],
                    columns=['name'], aggfunc=np.sum, fill_value=np.nan)

In [35]:
#удаляем в таблице столбики-дубликаты столбиков из result
# drop_columns=pivot.columns.str.contains('SE-')==True

# droped_columns = []
# for i in range(len(drop_columns)):
#     if drop_columns[i]==True:
#         droped_columns.append(pivot.columns[i])

# pivot=pivot.drop(droped_columns, axis=1)
# pivot.head()

In [36]:
#Доавляем столбики по операциям к итоговой таблице

result = result.merge(pivot, how = 'left', on =['modeliN', 'ros'])
result = result.drop(['Days','name'],axis=1)
result = result.drop_duplicates()

# Удаление моделей, превышающих ограничения

In [37]:
#Удаляем модели из drop_model_list

result = result.loc[~(result.modeliN.isin(drop_model_list)), : ] 
result=result.loc[~((result.Sewing.isna())&(result.Cutting.isna()))]
result = result.reset_index(drop=True)

# Подсчет СС с учетом страны

In [38]:
#Находим страну Sewing
cc_model = result[['ros','Sewing','modeliN']]
cc_model = cc_model.dropna()
cc_model['main_model'] = cc_model.modeliN
cc_model = cc_model.drop(['Sewing'], axis =1)
result = result.merge(cc_model, how = 'left', on = ['ros','modeliN'])


#Определяем страну для моделей без пошива
cc_no_sewing=result[['Sewing','modeliN']]
cc_no_sewing = cc_no_sewing.fillna(0)
cc_no_sewing = cc_no_sewing.groupby('modeliN')['Sewing'].sum().reset_index()
cc_no_sewing = cc_no_sewing[cc_no_sewing.Sewing==0].reset_index(drop=True)
cc_no_sewing['main_model_2'] = cc_no_sewing['modeliN']
cc_no_sewing = cc_no_sewing.drop(['Sewing'], axis =1)
result = result.merge(cc_no_sewing, how = 'left', on = ['modeliN'])

#Сливаем данные по main_model
result['main_model']=np.where(result.main_model_2.notna(), result.main_model_2, result.main_model)
result = result.drop(['main_model_2'], axis =1)

In [39]:
result.isna().sum()

ros                      0
modeliN                  0
dnp                      0
zakaz                    0
Dep                      0
Cat                      0
subCat                   0
Тип                      0
Target group             0
Category Production      0
Automated Machines     356
Cutting                  0
Embroidery             391
Finishing               24
Post Sewing            379
Printing               284
Production Total         0
SE-Attrition           423
SE-Brush               436
SE-Glove               438
SE-Laser               417
SE-Mesh                436
SE-Pinks               433
SE-Silicone            437
SE-Spray               411
Sewing                   0
Washing                285
post-Sewing             24
pre-Sewing               0
main_model               0
dtype: int64

# Дополнительные преобразования

In [40]:
#Делим категории на denim/non-denim
result['Category Production'] = result['Category Production'].str.lower()

new_cat=[]
for i in range(len(result)):
    if result['Category Production'][i]=='denim':
        new_cat.append('denim')
    else:
        new_cat.append('non-denim')


result['Category+2'] = new_cat

In [42]:
#Месяц и год
result['dnp']=pd.to_datetime(result['dnp'])
result['Month'] = result['dnp'].dt.strftime('%m')
result['Year'] = result['dnp'].dt.strftime('%Y')
result['quarter_FSD'] = result['dnp'].dt.quarter.apply(str)+'Q'+result.Year.str.slice(start=2)
result = result.drop_duplicates()

#Вывожу часть результата
result[['ros', 'modeliN', 'Dep', 'Cat', 'post-Sewing', 'pre-Sewing']].head()

,ros,modeliN,Dep,Cat,post-Sewing,pre-Sewing
0,Россия,BAC010149 серый меланж,Activewear,Act Pants,21.0,16.0
1,Россия,BJN009749 медиум-лайт/айс,Jeans,Jeans Commercial,41.0,12.0
2,Россия,BJN010234 черный,Jeans,Jeans Commercial,47.0,9.0
3,Россия,BJN010256 медиум-лайт/айс,Jeans,Jeans Commercial,24.0,16.0
4,Россия,BJN010435 серый,Jeans,Jeans Commercial,25.0,10.0


In [42]:
#Удаляем модели в кросс-странах, если их мало
if len(result[result.ros=='Россия+Украина'])<3:
    result = result[result.ros!='Россия+Украина']

# Выгрузка базы

In [43]:
#Выгружаем таблицу

result.to_excel("Lead_Time_Report.xlsx", index=False)